In [58]:
import scipy
import math
import numpy as np
import sys

import pymunk
from pymunk.pygame_util import *
from pymunk.vec2d import Vec2d
import pygame
from pygame.locals import *

In [64]:
#Pygame Variable/constant
size = 800
FPS = 50
GRAY = (220, 220, 220)
RED = (255, 0, 0)
BLACK = (0,0,0)
WHITE = (255,255,255)


#Simulation constant
stiffness = 100000
damping = 750

In [69]:
#Simluation Space
space = pymunk.Space()
space.gravity = 0,-450
space.damp = 0.5


#Object class
class Segment():    
    def __init__(self,x,y,x2,y2):
        self.x = x
        self.y = y
        self.x2 = x2
        self.y2 = y2
        self.body = pymunk.Body(body_type=pymunk.Body.STATIC)
        self.shape = pymunk.Segment(self.body,(x,y),(x2,y2),5)
        self.shape.elasticity = 0
        self.shape.friction = 0
        space.add(self.body,self.shape)
        
    def draw(self,screen):
        pygame.draw.line(screen,(0,0,0),(self.x,size-self.y),(self.x2,size-self.y2),5)

class Particle():
    def __init__(self,x,y):
        self.body = pymunk.Body()
        self.body.position = x, y
        self.shape = pymunk.Circle(self.body,5)
        self.shape.density = 0.5
        self.shape.elasticity = 1
        self.shape.friction = 10
        space.add(self.body,self.shape)
        
    def draw(self,screen):
        pygame.draw.circle(screen, (255,0,0), convert_position(self.body.position), 5)
            
        
class DampedSpring():
    def __init__(self, body1, attachment):
        self.body1 = body1
        self.body2 = attachment
        rl = self.body1.position.get_distance(self.body2.position) * 0.9
        joint = pymunk.DampedSpring(self.body1, self.body2, (0, 0), (0, 0), rl, stiffness, damping)
        joint.max_bias = 100
        space.add(joint)
    
    def draw(self,screen):
        pos1 = convert_position(self.body1.position)
        pos2 = convert_position(self.body2.position)
        pygame.draw.line(screen, BLACK, pos1, pos2, 5)
        
# Create a shape using particle & damped spring
class Square():
   
    def __init__(self):
        
        # PARTICLES MAKING SQUARE
        # we can make it auto x,y here #optional fix later
        self.Particles = []
        self.Particles.append(Particle(350,600))#0
        self.Particles.append(Particle(450,600)) #1
        self.Particles.append(Particle(350,500))#2
        self.Particles.append(Particle(450,500))#3
        
        # CONSTRAINTS = DAMPED SPRING CONSTRAINTS CONNECTING THE PARTICLES
        self.Springs = []
        self.Springs.append(DampedSpring(self.Particles[0].body,self.Particles[1].body))
        self.Springs.append(DampedSpring(self.Particles[0].body,self.Particles[2].body))
        self.Springs.append(DampedSpring(self.Particles[0].body,self.Particles[3].body))
        self.Springs.append(DampedSpring(self.Particles[2].body,self.Particles[1].body))
        self.Springs.append(DampedSpring(self.Particles[2].body,self.Particles[3].body))
        self.Springs.append(DampedSpring(self.Particles[1].body,self.Particles[3].body))
        
    def draw(self,screen):
        for particle in self.Particles:
            particle.draw(screen)
        for spring in self.Springs:
            spring.draw(screen)
        
class Circle():
    
    #idea : loop set of points and make particle & join them tgt
    # radius = radius of circle
    # n = number of points
    def __init__(self,radius,n):

        self.points = PointsInCircum(radius,n)
        self.Particles = []
        self.Springs = []
        
        #100,200 for offset (spawn points)
        for i in range(n):
            x,y = self.points[i][0],self.points[i][1]
            self.Particles.append(Particle(100+x,200+y))
            
        for i in range(n-1):
            self.Springs.append(DampedSpring(self.Particles[i].body,self.Particles[i+1].body))
        self.Springs.append(DampedSpring(self.Particles[-1].body,self.Particles[0].body))

        lenght = self.points[1][1]
        self.Square_Particles = []
        self.Square_Particles.append(Particle(100+lenght,200)) #0
        self.Square_Particles.append(Particle(100,200+lenght)) #1
        self.Square_Particles.append(Particle(100-lenght,200)) #2
        self.Square_Particles.append(Particle(100,200-lenght)) #3
        self.Square_Particles.append(Particle(100,200)) #<---- center point
              
        self.Springs.append(DampedSpring(self.Square_Particles[0].body,self.Square_Particles[1].body))
        self.Springs.append(DampedSpring(self.Square_Particles[1].body,self.Square_Particles[2].body))
        self.Springs.append(DampedSpring(self.Square_Particles[2].body,self.Square_Particles[3].body))
        self.Springs.append(DampedSpring(self.Square_Particles[3].body,self.Square_Particles[0].body))
        
        self.Springs.append(DampedSpring(self.Square_Particles[0].body,self.Square_Particles[4].body))
        self.Springs.append(DampedSpring(self.Square_Particles[1].body,self.Square_Particles[4].body))
        self.Springs.append(DampedSpring(self.Square_Particles[2].body,self.Square_Particles[4].body))
        self.Springs.append(DampedSpring(self.Square_Particles[3].body,self.Square_Particles[4].body))
        
        self.Springs.append(DampedSpring(self.Particles[0].body,self.Square_Particles[0].body))
        self.Springs.append(DampedSpring(self.Particles[1].body,self.Square_Particles[0].body))
        self.Springs.append(DampedSpring(self.Particles[1].body,self.Square_Particles[1].body))
        self.Springs.append(DampedSpring(self.Particles[2].body,self.Square_Particles[1].body))
        self.Springs.append(DampedSpring(self.Particles[3].body,self.Square_Particles[1].body))
        self.Springs.append(DampedSpring(self.Particles[3].body,self.Square_Particles[2].body))
        self.Springs.append(DampedSpring(self.Particles[4].body,self.Square_Particles[2].body))
        self.Springs.append(DampedSpring(self.Particles[5].body,self.Square_Particles[2].body))
        self.Springs.append(DampedSpring(self.Particles[5].body,self.Square_Particles[3].body))
        self.Springs.append(DampedSpring(self.Particles[6].body,self.Square_Particles[3].body))
        self.Springs.append(DampedSpring(self.Particles[7].body,self.Square_Particles[3].body))
        self.Springs.append(DampedSpring(self.Particles[7].body,self.Square_Particles[0].body))
        
    def draw(self,screen):
        for particle in self.Particles:
            particle.draw(screen)
        for spring in self.Springs:
            spring.draw(screen)
        for square_particle in self.Square_Particles:
            square_particle.draw(screen)
            
class SimulationSegment():
    
    # we create static shapes and put them in a list
    def __init__(self):
        self.Segments = []
        self.Segments.append(Segment(0,50,size,50))
        self.Segments.append(Segment(0,0,0,size))
        self.Segments.append(Segment(size,0,size,size))
         
    def draw(self,screen):
        for segment in self.Segments:
            segment.draw(screen)     
# custom functions
#
#
#function to convert from normal x,y to pygame
def convert_position(point):
    return int(point[0]), int(size-point[1])

# function to find distance of mouse_event and particles
def distance(event,particle):
#     print("------XY------")
#     print(abs(event[0] - particle[0]))
#     print(abs(event[1] - particle[1]))
    return abs(event[0] - particle[0]) < 20 and abs(event[1] - particle[1]) < 20 

# function to create circle: set of points with n size, with radius = r
def PointsInCircum(r,n=100):
    pi = math.pi
    return [(math.cos(2*pi/n*x)*r,math.sin(2*pi/n*x)*r) for x in range(0,n+1)]





#Simulation class
class Simulation():
    
    def __init__(self):
        pygame.init()
        self.running = True
        self.screen = pygame.display.set_mode((size,size))
        self.clock = pygame.time.Clock()
        self.pulling = False
        self.active_shape = None
        self.selected_shapes = []
        
        # SETUP #ADD SHAPES TO SIMULATION HERE #Circle(75,8)
        self.Shapes = [Circle(75,8),Square()]
        self.Segments = SimulationSegment()
        pygame.display.set_caption('Soft-Body Simulation (Square)')
    
    def do_event(self, event):
        
        if event.type == QUIT:
            self.running = False 
            
        elif event.type == MOUSEMOTION:
            self.p = event.pos
            
        # P: Position of the mouse when click    
        elif event.type == MOUSEBUTTONDOWN:
            p = from_pygame(event.pos, self.screen)
            self.active_shape = None
            fixed_pos = (event.pos[0],size-event.pos[1])
            for s in space.shapes:
                query = s.point_query(p)
                point = query.point
                if distance(fixed_pos,point):
                    self.active_shape = s
                    self.pulling = True
                    s.body.angle = (p - s.body.position).angle
                    if pygame.key.get_mods() and KMOD_META:
                        self.selected_shapes.append(s)
                    else:
                        self.selected_shapes = []         
        elif event.type == MOUSEBUTTONUP:
            if self.pulling:
                self.pulling = False
                b = self.active_shape.body
                p0 = Vec2d(b.position.x,b.position.y)
                p1 = from_pygame(event.pos, self.screen)
                impulse = 100 * (p0 - p1).rotated(b.angle)
                b.apply_impulse_at_local_point(impulse)        
        
        
        
    def run(self):
        while self.running:
            for event in pygame.event.get():
                self.do_event(event)
            self.draw()
            self.clock.tick(FPS)
            space.step(1/FPS)
        pygame.quit()
        
        
    # you can have better draw function bro this is too dumb
    def draw(self):
        self.screen.fill(GRAY)
        for shape in self.Shapes:
            shape.draw(self.screen)
        self.Segments.draw(self.screen)
        pygame.display.update()
        
        
if __name__ == '__main__':
    Simulation().run()